In [2]:
import pandas as pd

In [3]:
df_results = pd.read_excel("results.xlsx")
df_gemeinden = pd.read_excel("Gemeinden_Graubünden.xlsx")
df_population = pd.read_excel("Bevölkerung_Kanton_Graubünden.xlsx", usecols="A,AA,AB,AC", header=1)

In [4]:
print(df_population)

                GDENAME  Alle betroffenen Personen  Alle betroffenen Frauen  \
0            GRAUBÜNDEN                     143955                  49684.5   
1           Vaz/Obervaz                       1890                    656.0   
2          Lantsch/Lenz                        365                    112.5   
3        Schmitten (GR)                        131                     38.0   
4          Albula/Alvra                        933                    297.0   
..                  ...                        ...                      ...   
97            Rongellen                         48                     15.0   
98      Zillis-Reischen                        320                     99.5   
99              Ferrera                         46                     18.0   
100           Rheinwald                        378                    125.5   
101  Muntogna da Schons                        245                     75.0   

     Alle am wahrscheinlichsten betroffenen Frauen 

In [5]:
df_gemeinden["GDEHISTID"].dtype

dtype('int64')

In [ ]:
min_fahrzeiten = df_results.groupby("GDENAME")["Fahrzeit in min"].min().reset_index()
min_fahrzeiten.columns = ["GDENAME", "MinFahrzeit"]
print(min_fahrzeiten)

            GDENAME  MinFahrzeit
0      Albula/Alvra           30
1            Andeer           16
2             Arosa           45
3             Avers           41
4    Bergün Filisur           34
..              ...          ...
95      Vaz/Obervaz           28
96           Zernez           32
97  Zillis-Reischen           12
98           Zizers           13
99             Zuoz           16

[100 rows x 2 columns]


In [7]:
df_gemeinden_subset = df_gemeinden[["GDEHISTID", "GDENAME"]]

In [8]:
df_min_fahrzeit = df_gemeinden_subset.merge(min_fahrzeiten, on="GDENAME", how="inner")
df_population_subset = df_population[["GDENAME", "Alle betroffenen Personen"]]
df_min_fahrzeit = df_min_fahrzeit.merge(df_population_subset, on="GDENAME", how="inner")

In [9]:
print(df_min_fahrzeit)

    GDEHISTID             GDENAME  MinFahrzeit  Alle betroffenen Personen
0       15695               Grono           25                       1140
1       15962         Vaz/Obervaz           28                       1890
2       15963        Lantsch/Lenz           24                        365
3       15964      Schmitten (GR)           26                        131
4       15967              Brusio           16                        733
..        ...                 ...          ...                        ...
95      16125           Rheinwald           30                        378
96      16586  La Punt Chamues-ch            9                        528
97      16598  Muntogna da Schons           24                        245
98      16610            Klosters           23                       3013
99      16658                Chur            5                      27660

[100 rows x 4 columns]


In [10]:
bins = list(range(0, 120, 10)) + [120, 260]
print(bins)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 260]


In [13]:
labels = [f"{i}-{i+9} Min" for i in bins[:-2]] + ["120+ Min"]
print(labels)

['0-9 km', '10-19 km', '20-29 km', '30-39 km', '40-49 km', '50-59 km', '60-69 km', '70-79 km', '80-89 km', '90-99 km', '100-109 km', '110-119 km', '120+ Min']


In [ ]:

labels = ['0-9 Min', '10-19 Min', '20-29 Min', '30-39 Min', '40-49 Min', '50-59 Min', '60-69 Min', '70-79 Min', '80-89 Min', '90-99 Min', '100-109 Min', '110-119 Min', '120+ Min']
df_min_fahrzeit["FahrzeitKlasse"] = pd.cut(df_min_fahrzeit["MinFahrzeit"], bins=bins, labels=labels, right=False)
df_min_fahrzeit = df_min_fahrzeit.sort_values(by="FahrzeitKlasse")
df_grouped = df_min_fahrzeit.groupby("FahrzeitKlasse", as_index=False, observed=False)["Alle betroffenen Personen"].sum()

In [117]:
print(df_min_fahrzeit)

    GDEHISTID      GDENAME  MinFahrzeit  Alle betroffenen Personen  \
99      16658         Chur            5                      27660   
61      16036      Schiers            1                       2087   
69      16047        Davos            3                       7729   
72      16051        Cazis            5                       1712   
75      16054       Grüsch            5                       1526   
..        ...          ...          ...                        ...   
29      16000      Samnaun           40                        584   
18      15984        Avers           41                         99   
66      16042     Tujetsch           49                        781   
73      16052    Bregaglia           50                       1051   
71      16049  Val Müstair           73                        933   

   FahrzeitKlasse  
99        0-9 Min  
61        0-9 Min  
69        0-9 Min  
72        0-9 Min  
75        0-9 Min  
..            ...  
29      40-49 Min  

In [118]:
print(df_grouped)

   FahrzeitKlasse  Alle betroffenen Personen
0         0-9 Min                      60288
1       10-19 Min                      53432
2       20-29 Min                      15538
3       30-39 Min                       8409
4       40-49 Min                       4113
5       50-59 Min                       1051
6       60-69 Min                          0
7       70-79 Min                        933
8       80-89 Min                          0
9       90-99 Min                          0
10    100-109 Min                          0
11    110-119 Min                          0
12       120+ Min                          0
